# Import

In [ ]:
# imoprt GYM stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# import helpers
import numpy as np
import random
import os

# import stables_baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Types of spaces

In [ ]:
Discrete(3).sample()  # gives value {0,1,2}

In [ ]:
Box(0, 1, shape=(3, 3)).sample()  # gives [3x3] with values in [0,1]

In [ ]:
Tuple((Discrete(3), Box(0, 1, shape=(3,)))).sample()

In [ ]:
Dict({
    'height': Discrete(2),
    'speed': Box(0, 100, shape=(1,)),
    'color': Box(0, 255, shape=(4,))
}).sample()

In [ ]:
MultiBinary(4).sample()

In [ ]:
MultiDiscrete([4, 2]).sample()

# Building an environment

- build an agent to give us the best shower possible
- random temperature 
    - reward if $\in [37, 39]$ degrees

In [ ]:
class ShowerEnv(Env):
    def __init__(self) -> None:
        self.action_space = Discrete(3)  # temp down/unchanged/up
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60  # seconds


    def step(self, action: int) -> tuple[float, float, bool, dict]:
        self.state += action-1   # apply temp adj
        self.shower_length -= 1  # decrease shower time
        reward = 1 if int(self.state) in range(37, 39) else -1
        done = self.shower_length <= 0
        info = {}

        return self.state, reward, done, info


    def reset(self) -> float:
        self.state = 38 + random.randint(-3, 3)
        self.shower_length = 60
        return self.state


    def render(self): pass

In [ ]:
env = ShowerEnv()

# Test Environment

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {episode}, Score: {score}")
env.close()

# Train Model

In [ ]:
log_path = os.path.join('../output', 'training', 'logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=100_000)

# Save & evaluate model

In [ ]:
save_path = os.path.join('../output', 'training', 'saved models', 'ppo_shower')
model.save(save_path)

In [ ]:
del model
model = PPO.load(save_path)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=100)